# Train Model

In order to perform collision avoidance, a model must be trained to detect if the path forward is ``free`` or ``blocked``.

In [2]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### Dataset Preperation

Before the model can be trained, the dataset needs to be prepared for use.

In [ ]:
# Container for the dataset.
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [ ]:
# Split dataset into training and test sets.
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

In [ ]:
# Create batches of shuffled images
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

### Neural Network

A convolutional neural network is needed for the data to be trained on.

In [ ]:
# Utilize a pretrained model.
model = models.resnet18(pretrained=True)

In [ ]:
# Fix the model for only the free/blocked classification.
model.fc = torch.nn.Linear(512, 2)

In [ ]:
# Move the model to the GPU for faster training.
device = torch.device('cuda')
model = model.to(device)

### Train

Update the weights/biases of the model to where it should accurately predect of the path forward is ``free`` or ``blocked``.

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

### TensorRT

Create an optimized verson of the model with TensorRT for faster inference.

In [4]:
from torch2trt import torch2trt

modelRT = torchvision.models.resnet18(pretrained=False)
modelRT.fc = torch.nn.Linear(512, 2)
modelRT = modelRT.cuda().eval().half()
modelRT.load_state_dict(torch.load('best_model.pth'))

data = torch.zeros((1, 3, 224, 224)).cuda().half()
model_trt = torch2trt(modelRT, [data], fp16_mode=True)
torch.save(model_trt.state_dict(), 'collision_avoidance.pth')